# Params

In [1]:
import pandas as pd
import numpy as np
import torch
import torch_geometric as pyg
from tqdm.auto import *

from diffgd.data import GraphDrawingData
from diffgd.datasets import  RomeDataset
from diffgd.metrics import Stress

batch_size = 8
lr = 0.001
decay = 0.998
train_size = 200
num_steps = 1000



## Dataset

In [2]:
device = "cpu"
for backend, device_name in {
    torch.backends.mps: "mps",
    torch.cuda: "cuda",
}.items():
    if backend.is_available():
        device = device_name

In [3]:
dataset = RomeDataset(
    index=pd.read_csv("assets/rome_index.txt", header=None)[0],
)
layouts = np.load("assets/layouts/pmds.npy", allow_pickle=True)

datalist = list(dataset)
for i, data in enumerate(datalist):
    data.pos = torch.tensor(layouts[i]).float()
    data.feats = torch.randn((data.pos.shape[0], 16)).to(device)
print(len(datalist))

train_loader = pyg.loader.DataLoader(datalist[:train_size], batch_size=batch_size, shuffle=True)
val_loader = pyg.loader.DataLoader(datalist[11000:], batch_size=batch_size, shuffle=False)
test_loader = pyg.loader.DataLoader(datalist[10000:11000], batch_size=batch_size, shuffle=False)

/Users/xuefengli/24fall/DeepAesthetic/.conda/lib/python3.10/site-packages/torch_geometric/data/dataset.py:213: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  if osp.exists(f)

Transform graphs:   0%|          | 0/11531 [00:00<?, ?it/s]

11531


# Helpers

In [4]:
def generate_init_pos(batch):
    # feats = torch.rand_like(batch.pos)
    # feats = torch.ones((batch.pos.shape[0], 16)).to(device)
    feats = torch.randn((batch.pos.shape[0], 16)).to(device)
    feats, pos = generate_init(
        feats=batch.feats,
        pos=batch.pos,
        apsp=batch.apsp_attr,
        edge_index=batch.perm_index,
        batch_index=batch.batch,
    )
    return feats, pos

def get_edge_features(all_pair_shortest_path):
    return torch.cat([
        all_pair_shortest_path[:, None],
        1 / all_pair_shortest_path[:, None].square()
    ], dim=-1)

def generate_init(feats, pos, apsp, edge_index, batch_index):
    return feats, rescale_by_stress(pos, apsp, edge_index, batch_index)
    
def get_edge_features(all_pair_shortest_path):
    return torch.cat([
        all_pair_shortest_path[:, None],
        1 / all_pair_shortest_path[:, None].square()
    ], dim=-1)

def rescale_by_stress(pos, apsp, edge_index, batch_index):
    src_pos, dst_pos = pos[edge_index[0]], pos[edge_index[1]]
    dist = (dst_pos - src_pos).norm(dim=1)
    u_over_d = dist / apsp
    scatterd_u_over_d_2 = pyg.utils.scatter(u_over_d ** 2, batch_index[edge_index[0]])
    scatterd_u_over_d = pyg.utils.scatter(u_over_d, batch_index[edge_index[0]])
    scale = scatterd_u_over_d_2 / scatterd_u_over_d
    return pos / scale[batch_index][:, None]

In [11]:
c = Stress()
def sample(model, batch, device, num_timesteps=1000):
    model.eval()
    with torch.no_grad():
        x = torch.randn_like(batch.pos).to(device)
        
        for i in reversed(range(num_timesteps)):
            t = torch.full((batch.batch_size,), i, device=device, dtype=torch.long)
            x = model.p_sample(x, t)
        
        return x
    
def train_diffusion(model, train_loader, optim, scheduler, device, num_epochs=20):
    for epoch in range(num_epochs):
        model.train()
        losses = []
        scores = []
        for batch_idx, batch in enumerate(tqdm(train_loader)):
            batch = batch.to(device)
            model.zero_grad()
            
            t = torch.randint(0, model.timesteps, (1,)).item()
            
            node_feats, init_pos = generate_init_pos(batch)
            
            loss = model.p_losses(
                x_start=init_pos,
                node_feat=node_feats,
                edge_index=batch.perm_index,
                edge_attr=get_edge_features(batch.apsp_attr),
                batch_index=batch.batch,
                t=t,
            )
            
            pred = model.sample(
                node_feat=node_feats,
                edge_index=batch.perm_index,
                edge_attr=get_edge_features(batch.apsp_attr),
                batch_index=batch.batch,
            )
            pred = rescale_by_stress(pred, batch.apsp_attr, batch.perm_index, batch.batch)
            # score = c(pred, batch.apsp_attr, batch.perm_index, batch.batch, batch.edge_pair_index)
            loss.backward()
            optim.step()
            losses.append(loss.item())
            # scores.append(score)
        
        scheduler.step()
        print(f'[Epoch {epoch}] Train Loss: {np.mean(losses)}, score = {np.mean(scores)}')



# Model

In [12]:
from diffgd.model import DiffusionModel

model = DiffusionModel(in_dim=16, out_dim=32, num_layers=4, edge_feat_dim=2, timesteps=200, device=device).to(device)
optim = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optim, step_size=10, gamma=0.1)
# print(model.sqrt_one_minus_alphas_cumprod.shape)
# Train the model
train_diffusion(model, train_loader, optim, scheduler, device, num_epochs=20)

for batch_idx, batch in enumerate(tqdm(test_loader)):
    node_feats, init_pos = generate_init_pos(batch)
    sampled_coords = model.sample(
        node_feat=node_feats,
        edge_index=batch.perm_index,
        edge_attr=get_edge_features(batch.apsp_attr),
        batch_index=batch.batch,
    )

EGNNBasicLayer.Config(dense=False, bn='pyg_batch_norm', act='leaky_relu', dp=0.0, residual=False, aggr='mean', root_weight=True, norm=True)
EGNNBasicLayer.Config(dense=False, bn='pyg_batch_norm', act='leaky_relu', dp=0.0, residual=False, aggr='mean', root_weight=True, norm=True)
EGNNBasicLayer.Config(dense=False, bn='pyg_batch_norm', act='leaky_relu', dp=0.0, residual=False, aggr='mean', root_weight=True, norm=True)
EGNNBasicLayer.Config(dense=False, bn='pyg_batch_norm', act='leaky_relu', dp=0.0, residual=False, aggr='mean', root_weight=True, norm=True)


  0%|          | 0/25 [00:00<?, ?it/s]

[Epoch 0] Train Loss: nan, score = nan


/Users/xuefengli/24fall/DeepAesthetic/.conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/xuefengli/24fall/DeepAesthetic/.conda/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


  0%|          | 0/25 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
import numpy as np
import torch

model = DiffusionModel(in_dim=16, out_dim=16, num_layers=4, edge_feat_dim=2, timesteps=1000, device='cpu').to('cpu')

init_pos = torch.rand((16, 2))
node_feats = torch.rand((16, 16))
perm_index = torch.ones((2, 240), dtype=torch.long)

edge_attr = torch.ones((240, 2))  # Creates a proper (240, 2) tensor
t = torch.randint(0, model.timesteps, (16,), device='cpu').long()

loss = model.p_losses(
    x_start=init_pos,
    node_feat=node_feats,
    edge_index=perm_index,
    edge_attr=edge_attr,
    batch_index=batch.batch,
    t=t
)

print(loss)


EGNNBasicLayer.Config(dense=False, bn='batch_norm', act='leaky_relu', dp=0.0, residual=False, aggr='mean', root_weight=True, norm=True)
EGNNBasicLayer.Config(dense=False, bn='batch_norm', act='leaky_relu', dp=0.0, residual=False, aggr='mean', root_weight=True, norm=True)
EGNNBasicLayer.Config(dense=False, bn='batch_norm', act='leaky_relu', dp=0.0, residual=False, aggr='mean', root_weight=True, norm=True)
EGNNBasicLayer.Config(dense=False, bn='batch_norm', act='leaky_relu', dp=0.0, residual=False, aggr='mean', root_weight=True, norm=True)


NameError: name 'batch' is not defined